In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


#Model Finetuning

In [2]:
!pip install -qU bitsandbytes transformers datasets accelerate loralib einops xformers
!pip install -q -U git+https://github.com/huggingface/peft.git

import os
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
model_id = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model =AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


Unused kwargs: ['load_4bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [4]:
model = prepare_model_for_kbit_training(model)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [5]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [6]:
def generate_prompt(data_point):
    return f"""
<Human>: {data_point["instruction"]} {data_point["input"]}
<AI>: {data_point["output"]}
    """.strip()

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    # Tokenize the generated prompt
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

from datasets import load_dataset

# Load your dataset
dataset_name = 'amrachraf/arXiv-full-text-synthetic-instruct-tune'
dataset = load_dataset(dataset_name, split="train")



# Now the dataset will be tokenized and ready for fine-tuning



README.md:   0%|          | 0.00/547 [00:00<?, ?B/s]

(…)-00000-of-00001-d9a650f63b16925f.parquet:   0%|          | 0.00/15.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10861 [00:00<?, ? examples/s]

In [7]:
len(dataset)

10861

In [8]:
dataset = dataset.select(range(500))

In [9]:
# Shuffle and apply the tokenization function
dataset = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
training_args = transformers.TrainingArguments(
    auto_find_batch_size=True,
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=4,
    logging_steps=10,
    output_dir="/content/",
    save_strategy='epoch',
)

In [11]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.398500
20,2.232700
30,2.019800
40,2.191600
50,2.316100
60,2.115900
70,1.974400
80,1.818100
90,1.902100
100,2.052000


TrainOutput(global_step=500, training_loss=1.8749704933166504, metrics={'train_runtime': 9778.1604, 'train_samples_per_second': 0.051, 'train_steps_per_second': 0.051, 'total_flos': 1.761990695531136e+16, 'train_loss': 1.8749704933166504, 'epoch': 1.0})

In [12]:
model.save_pretrained("/content/drive/MyDrive/fine_tuned_falcon_7b_instruct_lora")

## RAG Workflow

In [13]:
!pip install -qU llama-index
!pip install -q llama-index-readers-papers
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-embeddings-huggingface
!pip install -q langchain-community
!pip install -q llama-index-readers-file

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.0 MB/s eta 0:00:00
ERROR: pi

In [14]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader, ServiceContext, StorageContext, load_index_from_storage
from llama_index.core.node_parser import SimpleNodeParser,get_leaf_nodes, get_root_nodes
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Settings

In [15]:
from llama_index.readers.papers import ArxivReader

loader = ArxivReader()
documents, abstracts = loader.load_papers_and_abstracts(
    search_query="Named Entity Disambiguation"
)

Failed to load file /content/.papers/61806030794aee5c2b4d4c64ab6f5bef.pdf with error: RetryError[<Future at 0x7f46d7a2a470 state=finished raised UnboundLocalError>]. Skipping...


In [ ]:
# documents[0]

In [16]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader, ServiceContext, StorageContext, load_index_from_storage
from llama_index.core.node_parser import SimpleNodeParser,get_leaf_nodes, get_root_nodes
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Settings


parser = SimpleNodeParser()

# Parse documents into nodes
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.embed_model = embed_model

In [ ]:
index = VectorStoreIndex.from_documents(documents)

index.set_index_id("vector_index")
index.storage_context.persist("/content/drive/My Drive/RAGgers/storage")

In [ ]:
# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="/content/drive/My Drive/RAGgers/storage")
# load index
index = load_index_from_storage(storage_context, index_id="vector_index")

In [ ]:
# !pip install -q llama-index-llms-llama-cpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
from peft import PeftModel

model_path = "/content/drive/MyDrive/RAGgers/fine_tuned_falcon_7b_instruct_lora"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b",  # Base Falcon model
    device_map="auto",  # Automatically map layers across available GPUs
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Load fine-tuned LoRA model
model = PeftModel.from_pretrained(base_model, model_path)

Unused kwargs: ['load_4bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
from typing import Any

from llama_index.core.callbacks import CallbackManager
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.core.llms.callbacks import llm_completion_callback


class ArXivLLM(CustomLLM):
    num_output: int = 2048
    model_name: str = "ArXivLLM"
    model: Any = None
    tokenizer: Any = None
    context_window: int = 2048  # Define model's context window size

    def __init__(self, model, tokenizer, num_output):
        super(ArXivLLM, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.num_output = num_output

    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            num_output=self.num_output,
            model_name=self.model_name,
        )

    def _get_available_context_size(self, prompt: str) -> int:
        """Calculate available context size based on the input prompt."""
        input_tokens = len(self.tokenizer(prompt)["input_ids"])
        available_tokens = self.context_window - input_tokens
        return available_tokens

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        # Calculate available context size
        available_tokens = self._get_available_context_size(prompt)

        # If available tokens are less than or equal to zero, raise an error
        if available_tokens <= 0:
            raise ValueError("Input exceeds the model's maximum context window.")

        # Adjust max_new_tokens to fit the available context size
        max_new_tokens = min(self.num_output, available_tokens)

        # Tokenize input
        inputs = self.tokenizer(prompt, truncation=True, return_tensors="pt", max_length=self.context_window)

        # Generate output
        outputs = self.model.generate(
            inputs.input_ids,
            max_length=len(inputs.input_ids[0]) + max_new_tokens,
            max_new_tokens=max_new_tokens,
            pad_token_id=self.tokenizer.pad_token_id
        )

        # Decode and return as text
        output_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return CompletionResponse(text=output_text)

    @llm_completion_callback()
    def stream_complete(self, prompt: str, **kwargs: Any) -> CompletionResponseGen:
        # Calculate available context size
        available_tokens = self._get_available_context_size(prompt)

        # If available tokens are less than or equal to zero, raise an error
        if available_tokens <= 0:
            raise ValueError("Input exceeds the model's maximum context window.")

        # Adjust max_new_tokens to fit the available context size
        max_new_tokens = min(self.num_output, available_tokens)

        # Tokenize input
        inputs = self.tokenizer(prompt, return_tensors="pt")

        # Generate output
        outputs = self.model.generate(
            inputs.input_ids,
            max_length=len(inputs.input_ids[0]) + max_new_tokens,
            max_new_tokens=max_new_tokens,
            pad_token_id=self.tokenizer.pad_token_id
        )
        output_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Stream tokens
        response = ""
        for token in output_text.split():  # Tokenize into words or subwords
            response += token + " "
            yield CompletionResponse(text=response.strip(), delta=token)


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in ArXivLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
tokenizer.pad_token = tokenizer.eos_token

custom_llm = ArXivLLM(model=model, tokenizer=tokenizer, num_output=128)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
Settings.llm = custom_llm


In [ ]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.core.retrievers import AutoMergingRetriever

# configure retriever
base_retriever = index.as_retriever(llm=custom_llm, similarity_top_k = 2)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

response_synthesizer = get_response_synthesizer(
            structured_answer_filtering=True,
            response_mode=ResponseMode.SIMPLE_SUMMARIZE,
        )
query_engine = RetrieverQueryEngine.from_args(retriever, response_synthesizer=response_synthesizer, llm = custom_llm)
response= query_engine.query("""How can we perform Named entity Disambiguation?""")
print(response.response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Both `max_new_tokens` (=93) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Context information is below.
---------------------
['page_label: 2\nTitle of this paper: Benchmarking Named Entity Disambiguation approaches for Streaming Graphs\nAuthors: Sutanay Choudhury, Chase Dowling\nDate published: 07/14/2014\nURL: http://arxiv.org/abs/1407.3751v1\n\n2 SUTANAY CHOUDHURY, CHASE DOWLING\n2. Background\nWithin in the broader ﬁeld of named entity disambiguation, we focus on Knowledge\nBase Population problem as introduced by the NIST Text Analytics Conference (TAC) [5].\nThis competition uses a 2008 version of Wikipedia as the Knowledge Base (KB). Given a\nstring referring to a person, organization or geo-political entity name and a background\ndocument providing contextual information, the entity linking task requires a system to\nreturn the ID of an entity in the knowledge base if it exists, else return NIL.\n2.1. Problem Statement. We have a knowledge base KB = (D,F,N ) where D, F and\nN represent a collection of documents, features and names. C contains informa